Goal of this notebook is to explore some of the secondary files in the dataset and see what value they might add

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from patsy import dmatrices
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from imblearn.over_sampling import SMOTE
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import KFold
%matplotlib inline

In [8]:
#Read the credit and application data sets
credit_card_balance_data = pd.read_csv('../Data/credit_card_balance.csv')
application_data = pd.read_csv('../Data/application_train.csv')

In [10]:
#Merge the data
merged_data = application_data.join(credit_card_balance_data, how='outer', on='SK_ID_CURR', lsuffix='application', rsuffix='cc_balance')